In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [5]:
pre_df = pd.read_csv('C:/Users/91909/Downloads/predictions (2).csv')
pre_df['combinations'] = pre_df['combinations'].str.split(' - ').apply(tuple)
pre_df.head()

,reviews,combinations,predicted_labels
0,Why does it look like someone spit on my food?...,"(price, positive)",0
1,Why does it look like someone spit on my food?...,"(price, neutral)",0
2,Why does it look like someone spit on my food?...,"(price, negative)",0
3,Why does it look like someone spit on my food?...,"(price, conflict)",0
4,Why does it look like someone spit on my food?...,"(price, none)",1


In [20]:
pre_df.columns

Index(['reviews', 'combinations', 'predicted_labels'], dtype='object')

In [21]:
mc_df = pd.read_csv('C:/Users/91909/Documents/Term 3/Machine Learning/assignment/senti/Sentiment-Analysis-App/data/McDonald_s_Reviews.csv', encoding = 'latin-1' , index_col=0)
mc_df.columns

Index(['store_name', 'category', 'store_address', 'latitude ', 'longitude',
       'rating_count', 'review_time', 'review', 'rating'],
      dtype='object')

In [35]:
final_df = pre_df.merge(mc_df, left_on='reviews', right_on='review', how='left')[['reviews', 'combinations', 'predicted_labels', 'store_address', 'latitude ', 'longitude', 'review_time', 'rating']]
final_df.head(2)

,reviews,combinations,predicted_labels,store_address,latitude,longitude,review_time,rating
0,Why does it look like someone spit on my food?...,"(price, positive)",0,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,3 months ago,1 star
1,Why does it look like someone spit on my food?...,"(price, neutral)",0,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,3 months ago,1 star


In [31]:
import numpy as np

class ReviewPredictionMatrix:
    def __init__(self, aspects, sentiments):
        self.aspects = aspects
        self.sentiments = sentiments
        # Initialize a matrix of -1s
        self.matrix = np.full((len(aspects), len(sentiments)), 0)
        
    def update_entry(self, aspect, sentiment, prediction):
        # Update the matrix based on the provided aspect, sentiment, and prediction value.
        row_idx = self.aspects.index(aspect)
        col_idx = self.sentiments.index(sentiment)
        self.matrix[row_idx, col_idx] = prediction

    def update_from_list(self, list_of_predictions):
        """
        Updates the matrix from a list of predictions.
        :param list_of_predictions: List of format [(tuple: ( tuple : aspect, sentiment), binary_prediction)]
        """
        for (aspect, sentiment), prediction in list_of_predictions:
            self.update_entry(aspect, sentiment, prediction)
    
    def get_matrix(self):
        return self.matrix
    
    def visualize(self):
        # Visualize the matrix for this review, e.g., using a heatmap.
        pass

In [62]:
# maintaining both orders of aspects and sentiments
aspects = ['price', 'anecdotes', 'food', 'ambience', 'service']
sentiments = ['positive', 'neutral', 'negative', 'conflict', 'none']

def aggregate_to_matrix(series_of_tuples):
    
    # Create a matrix object
    matrix_obj = ReviewPredictionMatrix(aspects, sentiments)
    
    # Loop over the series and update the matrix
    for absa_tuple in series_of_tuples:
        matrix_obj.update_entry(absa_tuple[0][0], absa_tuple[0][1], absa_tuple[1])
    
    return matrix_obj.get_matrix()



In [63]:
# Step 1: Create new column with tuple
final_df['combo_label_pairs'] = list(zip(final_df['combinations'], final_df['predicted_labels']))

# Step 2: Group by 'reviews' and aggregate the tuples
aggregated_pairs = final_df.groupby('reviews')['combo_label_pairs'].apply(aggregate_to_matrix).reset_index()

# Step 3: Merge with the original dataframe
merged_with_aggregation = final_df.drop(columns=['combinations', 'predicted_labels', 'combo_label_pairs']).drop_duplicates()
final_result = merged_with_aggregation.merge(aggregated_pairs, on='reviews')

final_result.head()
# The 'combo_label_pairs' column in 'final_result' dataframe will now have lists of tuple

,reviews,store_address,latitude,longitude,review_time,rating,combo_label_pairs
0,Why does it look like someone spit on my food?...,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,3 months ago,1 star,"[[0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 1, 0..."
1,It'd McDonalds. It is what it is as far as the...,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,5 days ago,4 stars,"[[0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [1, 0, 0, 0..."
2,Made a mobile order got to the speaker and che...,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,5 days ago,1 star,"[[0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0..."
3,My mc. Crispy chicken sandwich was ï¿½ï¿½ï¿½ï¿...,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,a month ago,5 stars,"[[0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [1, 0, 0, 0..."
4,"I repeat my order 3 times in the drive thru, a...","13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,2 months ago,1 star,"[[0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 1, 0..."


In [71]:
# check sentiemnt distribution for each aspect for the first review
x = 1
final_df['combo_label_pairs'][5*(x-1):5*x]

0    ((price, positive), 0)
1     ((price, neutral), 0)
2    ((price, negative), 0)
3    ((price, conflict), 0)
4        ((price, none), 1)
Name: combo_label_pairs, dtype: object

In [72]:
# make sure each row matches:
final_result['combo_label_pairs'][0]

array([[0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0]])

In [73]:
%pwd

'c:\\Users\\91909\\Documents\\Term 3\\Machine Learning\\assignment\\senti\\Sentiment-Analysis-App\\research'

In [ ]:
final_result.to_csv('final_result.csv', index=False)

In [ ]:
def aggregate_matrices_for_location(location):
    subset = df[df['store_address'] == location]
    aggregate_matrix = np.sum(np.stack(subset['ABSA_matrix'].values), axis=0)
    return aggregate_matrix

def plot_location_heatmap(location):
    aggregate_matrix = aggregate_matrices_for_location(location)
    plt.figure(figsize=(10,6))
    sns.heatmap(aggregate_matrix, annot=True, cmap='YlGnBu', xticklabels=sentiments, yticklabels=aspects)
    plt.title(f"Sentiment Distribution for Location: {location}")
    st.pyplot()

def plot_aspect_distribution():
    # This requires some preprocessing to get the aspect and sentiment distribution
    aspect_counts = {aspect: [] for aspect in aspects}
    for _, row in df.iterrows():
        matrix = row['ABSA_matrix']
        for i, aspect in enumerate(aspects):
            total = np.sum(matrix[i])
            aspect_counts[aspect].append(total)
    
    aspect_df = pd.DataFrame(aspect_counts, index=sentiments).T
    aspect_df.plot(kind='bar', stacked=True, figsize=(10,6))
    plt.title("Aspect-wise Sentiment Distribution")
    plt.ylabel("Number of Reviews")
    st.pyplot()